# PGGS analytic demo: guided importance sampling variance reductionAim: validate that a guidance potential U(γ) used to bias proposals reduces importance-weight variance by ≥50% versus uniform, aligning with the PGGS variance reduction acceptance criterion from [CFT+ULCC Framework Implementation.pdf](../../CFT+ULCC%20Framework%20Implementation.pdf).The notebook computes exact variances under each proposal on a discrete path space and saves two figures. Optional Monte Carlo checks use a fixed RNG for determinism.

In [ ]:
import numpy as npimport matplotlib.pyplot as pltfrom pathlib import Pathnp.set_printoptions(suppress=True, precision=6)fig_dir = Path('spec-first/artifacts/figures')fig_dir.mkdir(parents=True, exist_ok=True)rng = np.random.default_rng(0)

In [ ]:
def normalize_probs(p):    p = np.asarray(p, dtype=float)    total = p.sum()    if total <= 0:        raise ValueError('Probabilities must sum to a positive value.')    return p / totaldef convolve_circular(x, kernel):    x = np.asarray(x, dtype=float)    k = np.asarray(kernel, dtype=float)    if k.sum() != 0:        k = k / k.sum()    offsets = np.arange(k.size) - (k.size // 2)    y = np.zeros_like(x)    for coeff, off in zip(k, offsets):        y += coeff * np.roll(x, -off)    return ydef softmax_neg(x, gamma=1.0):    x = np.asarray(x, dtype=float)    z = np.exp(-gamma * (x - x.min()))    return z / z.sum()

In [ ]:
M = 300i = np.arange(M)theta = 2 * np.pi * i / Mbase = 3.0 + 1.25 * (np.sin(2 * theta) + 0.5 * np.cos(3 * theta))slope = 0.6 * (i / M) ** 2S = base + slopeS = np.maximum(S, 0.0)w = np.exp(-S)f = np.sin(theta) + 0.3 * np.cos(2 * theta)

In [ ]:
q_uniform = np.full(M, 1.0 / M)kernel = np.array([0.25, 0.5, 0.25])S_smooth = convolve_circular(S, kernel)delta = 0.1 * np.cos(5 * theta)U = S_smooth + deltagamma = 1.0q_guided = softmax_neg(U, gamma=gamma)

In [ ]:
Z_true = w.sum()IW_uni = w / q_uniformmu_uni = np.sum(q_uniform * IW_uni)var_uni = np.sum(q_uniform * (IW_uni - mu_uni) ** 2)IW_guided = w / q_guidedmu_guided = np.sum(q_guided * IW_guided)var_guided = np.sum(q_guided * (IW_guided - mu_guided) ** 2)rho = var_guided / var_uni

In [ ]:
N = 2000idx_u = rng.choice(M, size=N, replace=True, p=q_uniform)iw_u = w[idx_u] / q_uniform[idx_u]zhat_u = iw_u.mean()fhat_u = np.sum(iw_u * f[idx_u]) / np.sum(iw_u)idx_g = rng.choice(M, size=N, replace=True, p=q_guided)iw_g = w[idx_g] / q_guided[idx_g]zhat_g = iw_g.mean()fhat_g = np.sum(iw_g * f[idx_g]) / np.sum(iw_g)svar_u = iw_u.var(ddof=0)svar_g = iw_g.var(ddof=0)se_zhat_u = float(np.sqrt(var_uni / N))se_zhat_g = float(np.sqrt(var_guided / N))

In [ ]:
plt.figure(figsize=(6, 4))bins = 40plt.hist(IW_uni, bins=bins, weights=q_uniform, alpha=0.6, label='Uniform q', color='tab:gray')plt.hist(IW_guided, bins=bins, weights=q_guided, alpha=0.6, label='Guided q', color='tab:green')plt.xlabel('Importance weight (w_i / q_i)')plt.ylabel('Weighted frequency')plt.title('Weight distributions under proposals (exact, q-weighted)')plt.legend()plt.tight_layout()plt.savefig(fig_dir / 'pggs_weights_hist.png', dpi=200)plt.close()plt.figure(figsize=(5, 4))labels = ['Uniform', 'Guided']vars_exact = [var_uni, var_guided]bars = plt.bar(labels, vars_exact, color=['tab:gray', 'tab:green'])for bar in bars:    height = bar.get_height()    plt.text(bar.get_x() + bar.get_width()/2, height, f'{height:.3e}', ha='center', va='bottom', fontsize=8)plt.ylabel('Exact Var[w_i / q_i] under q')plt.title('Exact importance weight variance')plt.tight_layout()plt.savefig(fig_dir / 'pggs_variance.png', dpi=200)plt.close()

In [ ]:
print(f'Exact Z (mu_uni)     = {mu_uni:.12f}')print(f'Exact Z (mu_guided)  = {mu_guided:.12f}')print(f'Exact Var (uniform)  = {var_uni:.6e}')print(f'Exact Var (guided)   = {var_guided:.6e}')print(f'Variance reduction ρ = {rho:.6f}')print(f'MC: Zhat_uni {zhat_u:.6f} ± {se_zhat_u:.6f}; E[f]_IS_uni {fhat_u:.6f}')print(f'MC: Zhat_g   {zhat_g:.6f} ± {se_zhat_g:.6f}; E[f]_IS_g   {fhat_g:.6f}')assert rho <= 0.5, f'Variance reduction insufficient: ρ={rho:.3f} > 0.5'assert abs(mu_guided - mu_uni)/max(1e-12, abs(mu_uni)) < 1e-10, 'Exact means mismatch'print('Acceptance check passed: guided proposal reduces weight variance by ≥50%.')

Results summary: exact-variance computation avoids randomness while optional MC checks (fixed RNG seed) confirm trends and consistency with PGGS expectations.Artifacts saved on execution:- spec-first/artifacts/figures/pggs_weights_hist.png- spec-first/artifacts/figures/pggs_variance.png